In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, MiniBatchKMeans, Birch, OPTICS
import tensorflow as tf
import re

In [8]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.mixture import GaussianMixture
import random

# Membaca dataset
df = pd.read_csv('/content/drive/MyDrive/DATASET/DatasetBangkit_Capstone.csv')

# Inisialisasi objek LabelEncoder
label_encoder_waktu = LabelEncoder()
label_encoder_jenis = LabelEncoder()
label_encoder_rasa = LabelEncoder()

# Melakukan encoding untuk kolom 'waktu', 'jenis makanan', dan 'rasa makanan'
df['waktu'] = label_encoder_waktu.fit_transform(df['waktu'])
df['jenis makanan'] = label_encoder_jenis.fit_transform(df['jenis makanan'])
df['rasa makanan'] = label_encoder_rasa.fit_transform(df['rasa makanan'])

# Menampilkan nilai unik yang sudah diencode untuk setiap kolom
print("Nilai unik dalam kolom 'waktu':", df['waktu'].unique())
print("Nilai unik dalam kolom 'jenis makanan':", df['jenis makanan'].unique())
print("Nilai unik dalam kolom 'rasa makanan':", df['rasa makanan'].unique())

# Menyiapkan fitur dan label
X = df[['waktu', 'jenis makanan', 'rasa makanan']]
y = df[['makanan rekomendasi', 'protein_(g)', 'karbohidrat_(g)', 'tingkat bahaya']]

# Standarisasi data fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Melatih model Gaussian Mixture
gmm = GaussianMixture(n_components=5, random_state=42)
gmm.fit(X_scaled)

# Menambahkan kolom cluster ke dataframe
df['Cluster'] = gmm.predict(X_scaled)

# Definisi model menggunakan TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # Output layer dengan 5 neuron untuk 5 cluster
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Melatih model
model.fit(X_scaled, df['Cluster'], epochs=50, verbose=0)

# Variabel untuk menyimpan rekomendasi makanan sebelumnya untuk setiap waktu makan
rekomendasi_sebelumnya = {'Pagi': None, 'Siang': None, 'Malam': None}

# Fungsi untuk memberikan rekomendasi berdasarkan input pengguna
def rekomendasi_makanan(waktu_makanan, jenis_makanan, rasa_makanan):
    global rekomendasi_sebelumnya

    # Mengencode input pengguna
    waktu_makanan_enc = label_encoder_waktu.transform([waktu_makanan])[0]
    jenis_makanan_enc = label_encoder_jenis.transform([jenis_makanan])[0]
    rasa_makanan_enc = label_encoder_rasa.transform([rasa_makanan])[0]

    # Membuat vektor fitur dari input pengguna
    user_input = [[waktu_makanan_enc, jenis_makanan_enc, rasa_makanan_enc]]
    user_input_scaled = scaler.transform(user_input)

    # Menggunakan model untuk memprediksi cluster
    predicted_cluster = model.predict(user_input_scaled)
    cluster = tf.argmax(predicted_cluster, axis=1).numpy()[0]

    # Mendapatkan rekomendasi dari cluster yang diprediksi
    recommendations = df[df['Cluster'] == cluster]

    if not recommendations.empty:
        print(f"\nRekomendasi makanan untuk {waktu_makanan}:")
        if waktu_makanan != 'Malam':
            # Ambil rekomendasi yang belum pernah direkomendasikan sebelumnya
            if rekomendasi_sebelumnya[waktu_makanan] is None:
                filtered_recommendations = recommendations[(recommendations['jenis makanan'] == jenis_makanan) & (recommendations['rasa makanan'] == rasa_makanan)]
                if not filtered_recommendations.empty:
                    row = random.choice(filtered_recommendations.to_dict('records'))
                else:
                    row = random.choice(recommendations.to_dict('records'))
            else:
                unique_recommendations = recommendations[~recommendations['makanan rekomendasi'].isin(rekomendasi_sebelumnya[waktu_makanan])]
                filtered_recommendations = unique_recommendations[(unique_recommendations['jenis makanan'] == jenis_makanan) & (unique_recommendations['rasa makanan'] == rasa_makanan)]
                if not filtered_recommendations.empty:
                    row = random.choice(filtered_recommendations.to_dict('records'))
                else:
                    row = random.choice(unique_recommendations.to_dict('records'))
            rekomendasi_sebelumnya[waktu_makanan] = [row['makanan rekomendasi']]
        else:
            # Ambil rekomendasi secara acak
            row = random.choice(recommendations.to_dict('records'))
        print(f"\nMenu Makanan: {row['makanan rekomendasi']}")
        print(f"Tingkat Keasaman: {row['tingkat keasamaan']}")
        print(f"Protein: {row['protein_(g)']}")
        print(f"Karbohidrat: {row['karbohidrat_(g)']}")
        print(f"Tingkat Bahaya: {row['tingkat bahaya']}")
    else:
        print(f"\nMaaf, tidak ada rekomendasi yang tersedia untuk kriteria {waktu_makanan} yang dipilih.")

import ipywidgets as widgets

# Fungsi untuk input gejala dan keluhan menggunakan widget interaktif
def input_gejala():
    print("Pilih gejala dan keluhan yang dialami:")
    gejala_options = ['Rasa terbakar di dada/ulu hati', 'Rasa mual/muntah', 'Rasa kembung/penuh', 'Sakit perut/keram perut', 'Kesulitan menelan makanan/cairan']
    gejala_selected = widgets.SelectMultiple(options=gejala_options)
    return gejala_selected

# Fungsi untuk input pola makan menggunakan widget interaktif
def input_pola_makan():
    print("Pilih pola makan yang dialami:")
    pola_options = ['Sering konsumsi makanan pedas', 'Sering konsumsi makanan asam', 'Konsumsi makanan berlemak tinggi', 'Merokok']
    pola_selected = widgets.SelectMultiple(options=pola_options)
    return pola_selected

# Fungsi untuk meminta input obesitas menggunakan widget interaktif
def input_obesitas():
    obesitas_option = widgets.RadioButtons(options=['Ya', 'Tidak'], description='Apakah Anda obesitas?')
    return obesitas_option

# Menerima input dari pengguna
gejala_keluhan = input_gejala()
pola_makan = input_pola_makan()
obesitas = input_obesitas()

# Cetak hasil input
print("Gejala dan Keluhan yang Dipilih:", gejala_keluhan)
print("Pola Makan yang Dipilih:", pola_makan)
print("Apakah Anda Obesitas?:", obesitas.value)


# Fungsi untuk meminta input jenis makanan dan rasa makanan
def input_makanan(makanan):
    jenis = input(f"Makanan Apa Yang Akan Anda Makan Untuk {makanan} (panggang/kuah/goreng)? ").strip().capitalize()
    rasa = input(f"Rasa Makanan Untuk {makanan} (asin/gurih/pedas/netral/tawar/manis): ").strip().capitalize()
    return jenis, rasa

# Menerima input dari pengguna
jenis_makanan_sarapan, rasa_makanan_sarapan = input_makanan('Sarapan')
jenis_makanan_siang, rasa_makanan_siang = input_makanan('Makan Siang')
jenis_makanan_malam, rasa_makanan_malam = input_makanan('Makan Malam')

# Memanggil fungsi untuk memberikan rekomendasi makanan
print("\n\nRekomendasi Makanan:")
rekomendasi_makanan('Pagi', jenis_makanan_sarapan, rasa_makanan_sarapan)
rekomendasi_makanan('Siang', jenis_makanan_siang, rasa_makanan_siang)
rekomendasi_makanan('Malam', jenis_makanan_malam, rasa_makanan_malam)


Nilai unik dalam kolom 'waktu': [2 0 1]
Nilai unik dalam kolom 'jenis makanan': [2 1 0 3]
Nilai unik dalam kolom 'rasa makanan': [2 1 0]
Pilih gejala dan keluhan yang dialami:
Pilih pola makan yang dialami:
Gejala dan Keluhan yang Dipilih: SelectMultiple(options=('Rasa terbakar di dada/ulu hati', 'Rasa mual/muntah', 'Rasa kembung/penuh', 'Sakit perut/keram perut', 'Kesulitan menelan makanan/cairan'), value=())
Pola Makan yang Dipilih: SelectMultiple(options=('Sering konsumsi makanan pedas', 'Sering konsumsi makanan asam', 'Konsumsi makanan berlemak tinggi', 'Merokok'), value=())
Apakah Anda Obesitas?: Ya
Makanan Apa Yang Akan Anda Makan Untuk Sarapan (panggang/kuah/goreng)? kuah
Rasa Makanan Untuk Sarapan (asin/gurih/pedas/netral/tawar/manis): asin
Makanan Apa Yang Akan Anda Makan Untuk Makan Siang (panggang/kuah/goreng)? kuah
Rasa Makanan Untuk Makan Siang (asin/gurih/pedas/netral/tawar/manis): asin
Makanan Apa Yang Akan Anda Makan Untuk Makan Malam (panggang/kuah/goreng)? panggang
Ra

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 24ms/step

Rekomendasi makanan untuk Siang:

Menu Makanan: Burger keju
Tingkat Keasaman: Rendah
Protein: 15.0
Karbohidrat: 40.0
Tingkat Bahaya: Sedang
1/1 [==============================] - 0s 17ms/step

Rekomendasi makanan untuk Malam:

Menu Makanan: Manan samin
Tingkat Keasaman: Sedang
Protein: 3.4
Karbohidrat: 50.6
Tingkat Bahaya: Tinggi


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.mixture import GaussianMixture
import random

# Membaca dataset
df = pd.read_csv('/content/drive/MyDrive/DATASET/DatasetBangkit_Capstone.csv')

# Inisialisasi objek LabelEncoder
label_encoder_waktu = LabelEncoder()
label_encoder_jenis = LabelEncoder()
label_encoder_rasa = LabelEncoder()

# Melakukan encoding untuk kolom 'waktu', 'jenis makanan', dan 'rasa makanan'
df['waktu'] = label_encoder_waktu.fit_transform(df['waktu'])
df['jenis makanan'] = label_encoder_jenis.fit_transform(df['jenis makanan'])
df['rasa makanan'] = label_encoder_rasa.fit_transform(df['rasa makanan'])

# Menampilkan nilai unik yang sudah diencode untuk setiap kolom
print("Nilai unik dalam kolom 'waktu':", df['waktu'].unique())
print("Nilai unik dalam kolom 'jenis makanan':", df['jenis makanan'].unique())
print("Nilai unik dalam kolom 'rasa makanan':", df['rasa makanan'].unique())

# Menyiapkan fitur dan label
X = df[['waktu', 'jenis makanan', 'rasa makanan']]
y = df[['makanan rekomendasi', 'protein_(g)', 'karbohidrat_(g)', 'tingkat bahaya']]

# Standarisasi data fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Melatih model Gaussian Mixture
gmm = GaussianMixture(n_components=5, random_state=42)
gmm.fit(X_scaled)

# Menambahkan kolom cluster ke dataframe
df['Cluster'] = gmm.predict(X_scaled)

# Definisi model menggunakan TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # Output layer dengan 5 neuron untuk 5 cluster
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Melatih model
model.fit(X_scaled, df['Cluster'], epochs=50, verbose=0)

# Menyimpan model ke dalam file .h5
model.save('model_rekomendasi_makanan.h5')

print("Model telah disimpan sebagai model_rekomendasi_makanan.h5")

# Variabel untuk menyimpan rekomendasi makanan sebelumnya untuk setiap waktu makan
rekomendasi_sebelumnya = {'Pagi': None, 'Siang': None, 'Malam': None}

# Fungsi untuk memberikan rekomendasi berdasarkan input pengguna
def rekomendasi_makanan(waktu_makanan, jenis_makanan, rasa_makanan):
    global rekomendasi_sebelumnya

    # Mengencode input pengguna
    waktu_makanan_enc = label_encoder_waktu.transform([waktu_makanan])[0]
    jenis_makanan_enc = label_encoder_jenis.transform([jenis_makanan])[0]
    rasa_makanan_enc = label_encoder_rasa.transform([rasa_makanan])[0]

    # Membuat vektor fitur dari input pengguna
    user_input = [[waktu_makanan_enc, jenis_makanan_enc, rasa_makanan_enc]]
    user_input_scaled = scaler.transform(user_input)

    # Menggunakan model untuk memprediksi cluster
    predicted_cluster = model.predict(user_input_scaled)
    cluster = tf.argmax(predicted_cluster, axis=1).numpy()[0]

    # Mendapatkan rekomendasi dari cluster yang diprediksi
    recommendations = df[df['Cluster'] == cluster]

    if not recommendations.empty:
        print(f"\nRekomendasi makanan untuk {waktu_makanan}:")
        if waktu_makanan != 'Malam':
            # Ambil rekomendasi yang belum pernah direkomendasikan sebelumnya
            if rekomendasi_sebelumnya[waktu_makanan] is None:
                filtered_recommendations = recommendations[(recommendations['jenis makanan'] == jenis_makanan) & (recommendations['rasa makanan'] == rasa_makanan)]
                if not filtered_recommendations.empty:
                    row = random.choice(filtered_recommendations.to_dict('records'))
                else:
                    row = random.choice(recommendations.to_dict('records'))
            else:
                unique_recommendations = recommendations[~recommendations['makanan rekomendasi'].isin(rekomendasi_sebelumnya[waktu_makanan])]
                filtered_recommendations = unique_recommendations[(unique_recommendations['jenis makanan'] == jenis_makanan) & (unique_recommendations['rasa makanan'] == rasa_makanan)]
                if not filtered_recommendations.empty:
                    row = random.choice(filtered_recommendations.to_dict('records'))
                else:
                    row = random.choice(unique_recommendations.to_dict('records'))
            rekomendasi_sebelumnya[waktu_makanan] = [row['makanan rekomendasi']]
        else:
            # Ambil rekomendasi secara acak
            row = random.choice(recommendations.to_dict('records'))
        print(f"\nMenu Makanan: {row['makanan rekomendasi']}")
        print(f"Tingkat Keasaman: {row['tingkat keasamaan']}")
        print(f"Protein: {row['protein_(g)']}")
        print(f"Karbohidrat: {row['karbohidrat_(g)']}")
        print(f"Tingkat Bahaya: {row['tingkat bahaya']}")
    else:
        print(f"\nMaaf, tidak ada rekomendasi yang tersedia untuk kriteria {waktu_makanan} yang dipilih.")

import ipywidgets as widgets

# Fungsi untuk input gejala dan keluhan menggunakan widget interaktif
def input_gejala():
    print("Pilih gejala dan keluhan yang dialami:")
    gejala_options = ['Rasa terbakar di dada/ulu hati', 'Rasa mual/muntah', 'Rasa kembung/penuh', 'Sakit perut/keram perut', 'Kesulitan menelan makanan/cairan']
    gejala_selected = widgets.SelectMultiple(options=gejala_options)
    return gejala_selected

# Fungsi untuk input pola makan menggunakan widget interaktif
def input_pola_makan():
    print("Pilih pola makan yang dialami:")
    pola_options = ['Sering konsumsi makanan pedas', 'Sering konsumsi makanan asam', 'Konsumsi makanan berlemak tinggi', 'Merokok']
    pola_selected = widgets.SelectMultiple(options=pola_options)
    return pola_selected

# Fungsi untuk meminta input obesitas menggunakan widget interaktif
def input_obesitas():
    obesitas_option = widgets.RadioButtons(options=['Ya', 'Tidak'], description='Apakah Anda obesitas?')
    return obesitas_option

# Menerima input dari pengguna
gejala_keluhan = input_gejala()
pola_makan = input_pola_makan()
obesitas = input_obesitas()

# Cetak hasil input
print("Gejala dan Keluhan yang Dipilih:", gejala_keluhan)
print("Pola Makan yang Dipilih:", pola_makan)
print("Apakah Anda Obesitas?:", obesitas.value)


# Fungsi untuk meminta input jenis makanan dan rasa makanan
def input_makanan(makanan):
    jenis = input(f"Makanan Apa Yang Akan Anda Makan Untuk {makanan} (panggang/kuah/goreng)? ").strip().capitalize()
    rasa = input(f"Rasa Makanan Untuk {makanan} (asin/gurih/pedas/netral/tawar/manis): ").strip().capitalize()
    return jenis, rasa

# Menerima input dari pengguna
jenis_makanan_sarapan, rasa_makanan_sarapan = input_makanan('Sarapan')
jenis_makanan_siang, rasa_makanan_siang = input_makanan('Makan Siang')
jenis_makanan_malam, rasa_makanan_malam = input_makanan('Makan Malam')

# Memanggil fungsi untuk memberikan rekomendasi makanan
print("\n\nRekomendasi Makanan:")
rekomendasi_makanan('Pagi', jenis_makanan_sarapan, rasa_makanan_sarapan)
rekomendasi_makanan('Siang', jenis_makanan_siang, rasa_makanan_siang)
rekomendasi_makanan('Malam', jenis_makanan_malam, rasa_makanan_malam)


Nilai unik dalam kolom 'waktu': [2 0 1]
Nilai unik dalam kolom 'jenis makanan': [2 1 0 3]
Nilai unik dalam kolom 'rasa makanan': [2 1 0]


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model telah disimpan sebagai model_rekomendasi_makanan.h5
Pilih gejala dan keluhan yang dialami:
Pilih pola makan yang dialami:
Gejala dan Keluhan yang Dipilih: SelectMultiple(options=('Rasa terbakar di dada/ulu hati', 'Rasa mual/muntah', 'Rasa kembung/penuh', 'Sakit perut/keram perut', 'Kesulitan menelan makanan/cairan'), value=())
Pola Makan yang Dipilih: SelectMultiple(options=('Sering konsumsi makanan pedas', 'Sering konsumsi makanan asam', 'Konsumsi makanan berlemak tinggi', 'Merokok'), value=())
Apakah Anda Obesitas?: Ya
Makanan Apa Yang Akan Anda Makan Untuk Sarapan (panggang/kuah/goreng)? goreng
Rasa Makanan Untuk Sarapan (asin/gurih/pedas/netral/tawar/manis): asin
Makanan Apa Yang Akan Anda Makan Untuk Makan Siang (panggang/kuah/goreng)? goreng
Rasa Makanan Untuk Makan Siang (asin/gurih/pedas/netral/tawar/manis): manis
Makanan Apa Yang Akan Anda Makan Untuk Makan Malam (panggang/kuah/goreng)? goreng
Rasa Makanan Untuk Makan Malam (asin/gurih/pedas/netral/tawar/manis): gurih




/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 22ms/step

Rekomendasi makanan untuk Malam:

Menu Makanan: Mie celon
Tingkat Keasaman: Sedang
Protein: 3.0
Karbohidrat: 17.8
Tingkat Bahaya: Rendah


In [10]:
from google.colab import files

files.download('model_rekomendasi_makanan.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>